# Equipo: Foraneos y un Emi

## Carolina Arratia Camacho - A01367552

## Emiliano Mendoza Nieto - A01706083

## Frida Lizett Zavala Pérez - A01275226

## Fabián González Vera - A01367585

## Jazzareth Bernal Martínez- A01367882

## TC3007B
### Text Generation

<br>

### Simple LSTM Text Generator using WikiText-2

<br>

- Objective:
    - Gain a fundamental understanding of Long Short-Term Memory (LSTM) networks.
    - Develop hands-on experience with sequence data processing and text generation in PyTorch. Given the simplicity of the model, amount of data, and computer resources, the text you generate will not replace ChatGPT, and results must likely will not make a lot of sense. Its only purpose is academic and to understand the text generation using RNNs.
    - Enhance code comprehension and documentation skills by commenting on provided starter code.
    
<br>

- Instructions:
    - Code Understanding: Begin by thoroughly reading and understanding the code. Comment each section/block of the provided code to demonstrate your understanding. For this, you are encouraged to add cells with experiments to improve your understanding

    - Model Overview: The starter code includes an LSTM model setup for sequence data processing. Familiarize yourself with the model architecture and its components. Once you are familiar with the provided model, feel free to change the model to experiment.

    - Training Function: Implement a function to train the LSTM model on the WikiText-2 dataset. This function should feed the training data into the model and perform backpropagation.

    - Text Generation Function: Create a function that accepts starting text (seed text) and a specified total number of words to generate. The function should use the trained model to generate a continuation of the input text.

    - Code Commenting: Ensure that all the provided starter code is well-commented. Explain the purpose and functionality of each section, indicating your understanding.

    - Submission: Submit your Jupyter Notebook with all sections completed and commented. Include a markdown cell with the full names of all contributing team members at the beginning of the notebook.
    
<br>

- Evaluation Criteria:
    - Code Commenting (60%): The clarity, accuracy, and thoroughness of comments explaining the provided code. You are suggested to use markdown cells for your explanations.

    - Training Function Implementation (20%): The correct implementation of the training function, which should effectively train the model.

    - Text Generation Functionality (10%): A working function is provided in comments. You are free to use it as long as you make sure to uderstand it, you may as well improve it as you see fit. The minimum expected is to provide comments for the given function.

    - Conclusions (10%): Provide some final remarks specifying the differences you notice between this model and the one used  for classification tasks. Also comment on changes you made to the model, hyperparameters, and any other information you consider relevant. Also, please provide 3 examples of generated texts.



Instalación de paquetes y librerías para el procesamiento de lenguaje natural.

In [2]:
!pip install portalocker
!pip install --upgrade portalocker

import numpy as np
#PyTorch libraries
import torch
import torchtext
from torchtext.datasets import WikiText2
# Dataloader library
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
# Libraries to prepare the data
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
# neural layers
from torch import nn
from torch.nn import functional as F
import torch.optim as optim
from tqdm import tqdm

import random

Asigna a device la cadena 'cuda' si hay una GPU disponible, y 'cpu' si no hay GPU. Esta línea permite seleccionar automáticamente el dispositivo de procesamiento disponible en el sistema.

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Se carga automáticamente los conjuntos de datos de entrenamiento, validación y prueba del conjunto de datos WikiText-2.

In [4]:
train_dataset, val_dataset, test_dataset = WikiText2()

Primero se define un tokenizador el cual divide el texto en tokens básicos del inglés, y una función generadora que con el tokenizador creado anteriormente produce los tokens a partir de la data, a medida que se van necesitando para evitar cargarlos de una vez todos en la memoria.


In [5]:
tokeniser = get_tokenizer('basic_english')
def yield_tokens(data):
    for text in data:
        yield tokeniser(text)

Se construye un vocabuliario a partir de lso tokens generados en el conjunto de datos de entrenamiento. Se considera la generación de tokens especiales. ```("<unk>", "<pad>", "<bos>", "<eos>")```.

También se establece un índice predenterminado del vocabulario guíadose del token ```"<unk>"```. Para usarse en caso de encontrar algún token que no se encuentre en el vocabulario.

In [6]:
# Build the vocabulary
vocab = build_vocab_from_iterator(yield_tokens(train_dataset), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
#set unknown token at position 0
vocab.set_default_index(vocab["<unk>"])

Se genera la función ```data_process``` la cual prepara los datos para usarlo en el modelo posteriormente, en este caso el modelo de generación de texto.

In [7]:
seq_length = 50 #longitud de las secuencias

# Función para procesar datos
def data_process(raw_text_iter, seq_length = 50):
    # Tokeniza y convierte en tensores cada elemento del iterable de texto
    data = [torch.tensor(vocab(tokeniser(item)), dtype=torch.long) for item in raw_text_iter]

     # Combina los tensores en uno solo, eliminando tensores vacíos
    data = torch.cat(tuple(filter(lambda t: t.numel() > 0, data))) #remove empty tensors
#     target_data = torch.cat(d)

    # Crea conjuntos basados en el tensor combinado
    return (data[:-(data.size(0)%seq_length)].view(-1, seq_length),
            data[1:-(data.size(0)%seq_length-1)].view(-1, seq_length))

# Crea tensores para cada set de datos (validación, train y test)
x_train, y_train = data_process(train_dataset, seq_length)
x_val, y_val = data_process(val_dataset, seq_length)
x_test, y_test = data_process(test_dataset, seq_length)

Crear conjuntos de datos (`TensorDataset`) para entrenamiento, validación y prueba utilizando los tensores previamente procesados (`x_train`, `y_train`, `x_val`, `y_val`, `x_test`, `y_test`). Estos conjuntos están listos para ser utilizados con PyTorch para el entrenamiento y evaluación de modelos de lenguaje.

In [8]:
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)
test_dataset = TensorDataset(x_test, y_test)

Estos loaders se utilizan en el entrenamiento de modelos para proporcionar lotes de datos al modelo en cada iteración. La opción `shuffle=True` garantiza que los lotes se seleccionen de manera aleatoria en cada época, lo que es útil para mejorar la generalización del modelo.

In [9]:
batch_size = 64  # choose a batch size that fits your computation resources
                # definir el lote de procesamiento
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True) #carga datos al conjunto de entrenamiento y los combina,
                                                                                              #elimina el lote final si en menor que el batch_size
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True) #carga datos al conjunto de validación con las mismas
                                                                                          #configuraciones que el el conjunto de entrenamiento.
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)#carga datos al conjunto de prueba con las mismas
                                                                                          #configuraciones que el el conjunto de entrenamiento.

Se define la clase para el modelo LSTM, el cual se utiliza para procesar las secuencias de texto y poder predecir la palabra siguiente.

In [10]:
# Define the LSTM model
# Feel free to experiment
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()

        # capa de embedding para convertir índices de palabras en vectores densos
        self.embeddings = nn.Embedding(vocab_size, embed_size)

        # tamaño oculto de la capa LSTM y número de capas
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # capa LSTM
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        # capa de salida, transforma la salida al tamaño del vocabulario
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, text, hidden):
        embeddings = self.embeddings(text)
        output, hidden = self.lstm(embeddings, hidden)
        decoded = self.fc(output)
        #retorna la salida y el estado oculto
        return decoded, hidden

    def init_hidden(self, batch_size):
        # inicializa el estado oculto de la capa LSTM
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device),
                torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device))


#tamaño del vocabulario
vocab_size = len(vocab)
#tamaño del embeding
emb_size = 100
neurons = 128 # the dimension of the feedforward network model, i.e. # of neurons
num_layers = 2 # the number of nn.LSTM layers
#creear una instancia del modelo lstm
model = LSTMModel(vocab_size, emb_size, neurons, num_layers)


Definición de la función de entrenamineto. Esta función se ejecuta durante múltiples épocas y realiza la retropopágación y la actualización de parámetros para ajustar el modelo a los datos de entrenamiento.

In [11]:
def train(model, epochs, optimiser, loss_function, train_loader):
    # mueve el procesamineto del modelo a la GPU, si este está disponible.
    model = model.to(device=device)

    # establece el modelo en modo de entrenamiento
    model.train()

    # ciclo de entrenamiento por las épocas
    for epoch in range(epochs):
        total_loss = 0

        # inicializa el estado oculto antes de cada época
        hidden = model.init_hidden(batch_size)

        # itera sobre los lotes de datos de entrenamiento
        for i, (data, targets) in enumerate(train_loader):
            # se mueven los datos y etiquetas a la GPU
            data, targets = data.to(device), targets.to(device)

            # reinicia los gradientes
            optimiser.zero_grad()

            # Inicializa los estados ocultos para la primera entrada del lote
            hidden = tuple([each.data for each in hidden])

            # Propagación hacia adelante
            output, hidden = model(data, hidden)

            # Calcula la pérdida
            loss = loss_function(output.view(-1, vocab_size), targets.view(-1))
            total_loss += loss.item()

            # Backpropagation
            loss.backward()

            # Actualiza los parámetros
            optimiser.step()

        # Imprime información útil
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss/len(train_loader)}')



En la siguiente sección se entrena el modelo (se manda a llamar la función de entrenamiento), usando como optizador el  algoritmo Adam y cross entropy como función de pérdida.



In [12]:
# Call the train function
loss_function = nn.CrossEntropyLoss()# definición de la función de perdida
lr = 0.0005 # taza de aprendizaje
epochs = 50 # número de épocas
optimiser = optim.Adam(model.parameters(), lr=lr) # optimizador para ajustar los parámetros
train(model, epochs, optimiser, loss_function, train_loader)


Epoch 1/50, Loss: 7.071770977973938
Epoch 2/50, Loss: 6.5374506033957
Epoch 3/50, Loss: 6.287883224338293
Epoch 4/50, Loss: 6.104196494072676
Epoch 5/50, Loss: 5.972130925953389
Epoch 6/50, Loss: 5.871773736923933
Epoch 7/50, Loss: 5.789107673615217
Epoch 8/50, Loss: 5.717801616340876
Epoch 9/50, Loss: 5.655099970847369
Epoch 10/50, Loss: 5.598686393350363
Epoch 11/50, Loss: 5.547684736549854
Epoch 12/50, Loss: 5.500482453405857
Epoch 13/50, Loss: 5.456367491185665
Epoch 14/50, Loss: 5.415236841887236
Epoch 15/50, Loss: 5.377145034819842
Epoch 16/50, Loss: 5.342250720411539
Epoch 17/50, Loss: 5.309158261120319
Epoch 18/50, Loss: 5.277817095816135
Epoch 19/50, Loss: 5.247809153050184
Epoch 20/50, Loss: 5.219631105661392
Epoch 21/50, Loss: 5.192895857989788
Epoch 22/50, Loss: 5.166794969886541
Epoch 23/50, Loss: 5.14173918813467
Epoch 24/50, Loss: 5.1176565803587435
Epoch 25/50, Loss: 5.094218428432941
Epoch 26/50, Loss: 5.071738664060831
Epoch 27/50, Loss: 5.04964544698596
Epoch 28/50, 

Función para generar el texto, la cual recibe como párametrlos el modelo, el texto inicial, el número de palabras a generar y la temperatura, el cuál controla la aleatoriedad del proceso. Valores mayores a 1 hacen que sea más aleatoria, mientras que valores más próximos a 0 hacen que sea menos aleatorio.

Esta funcion itera sobre el número de palabras, con cada iteración se agrega una palabra a la secuencia, esta secuencia es usada como el input para la siguiente iteración.

In [13]:
def generate_text(model, start_text, num_words, temperature):
    model.eval() # Indica al modelo que se va a realizar una evaluacion
    words = tokeniser(start_text) # Genera tokens a partir del texto inicial
    hidden = model.init_hidden(1) # Inicializa el estado oculto de las capas

    for i in range(num_words):
        # Convierte la secuencia de palabras en tensores
        x = torch.tensor([[vocab[word] for word in words[-seq_length:]]], dtype=torch.long, device=device)
        y_pred, hidden = model(x, hidden) # Realiza forward propagation
        last_word_logits = y_pred[0][-1] # Obtiene el ultimo output
        p = F.softmax(last_word_logits / temperature, dim=0).detach().cpu().numpy() # Calcula la probabilidad
        word_index = np.random.choice(len(last_word_logits), p=p) # Selecciona las palabras
        words.append(vocab.lookup_token(word_index)) # Agrega palabra a la secuencia de palabras

    return ' '.join(words)

Ejemplos de textos generados cambiando el parámetro *temperature*.

In [14]:
print(generate_text(model, start_text="I like", num_words=100, temperature=1.0))
print(generate_text(model, start_text="Nice to meet you", num_words=100, temperature=1.0))
print(generate_text(model, start_text="You should", num_words=100, temperature=1.0))

i like changing calculations as a sign of a <unk> and you ultimately . but what leigh broke the school was prompted by german clark @-@ e , a boss comics video song , legends and chains . hurlford reach overseas with sea mark , as a partial churrigueresque engine . the brazen ibis cannot be considered one of the episodes contest from the end of the incident , he then enjoyed to don ' s reign , in 1946 . i represented a concert @-@ <unk> gown and conductor before reinforcements . owing to the creation of actors and he sent
nice to meet you never <unk> her 1769 that bird will rooted 22 copies during defense time . he was appointed and causes to the charity . fowler then won the album , he would have hot a dvd in the michael edition . the fifty @-@ time @-@ minute sister campaigns cross inquiry against belgium and his past father , although he regarded this influences there . contemporary international chapters wrote , with the medway megaliths bc , one picked reveal in this game judge an

In [15]:
print(generate_text(model, start_text="I don't like", num_words=100, temperature=0.5))

i don ' t like a <unk> . i . calospora , and i <unk> . he was <unk> in the second round of the same year , with the <unk> of the song . the album was released in the united states , and the album is a <unk> in the <unk> <unk> & <unk> . the final , which is the first time in the urn , the song was in the episode ' s music of the year and the second of the episode ' s final season . the album was released in the united states , and the third @-@


In [16]:
print(generate_text(model, start_text="Hello everyone", num_words=100, temperature=0.2))


hello everyone , the <unk> , and <unk> <unk> . = = = <unk> = = = the <unk> of the <unk> is a <unk> of the <unk> , a <unk> <unk> , and <unk> <unk> . the <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> ( <unk> <unk> ) , <unk> <unk> <unk> <unk> ( <unk> <unk> ) , <unk> <unk> ( <unk> <unk> ) , <unk> ( <unk> <unk> ) , <unk> ( <unk> <unk> ) , <unk> ( <unk> <unk> ) , <unk> ( <unk> <unk> )


Al generar las predicciones nos percatamos que el parámetro temperature afecta en gran medida la forma en la que estos se generan, observando un mejor comportamiento cuándo este es 1, debido a que le da más soltura al modelo de elegir entre las palabras seleccionadas, y no es tan determinista.

Este modelo se compone de 2 capas LSTM, 128 neuronas y un tamaño de embedding de 100. El learning rate es de 0.0005, usa el optimizador Adam, la función de pérdida es crossentropy y se entrenó con 50 épocas.

La principal diferencia entre este modelo y el usado para la tarea de clasificación es el tipo de red, este modelo usa una red LSTM y el modelo de clasifición usa una RNN. A pesar de que ambas redes tienen la capacidad de "recordar" los valores que recibieron anteriormente, las redes RNN tienen problemas con secuencias largas.

Las LSTM son un tipo de RNN, sin embargo puede retener mejor la información a largo plazo. Tienen una estructura más compleja que las RNN convencionales, la cual consiste en diversas entradas y salidas, una de las principales diferencias y al mismo tiempo ventajas de las LSTM, es que son capaces de retener o descartar selectivamente la información del estado oculto, esto es beneficioso para el problema de generación de texto ya que se puede olvidar de manera selectiva cierta información.
